# Reactor Kinetics Example 

Jialu Wang (jwang44@nd.edu) and Alex Dowling (adowling@nd.edu)

University of Notre Dame

This notebook conducts design of experiments for a reactor kinetics experiment with the Pyomo.DOE.
    

In [1]:
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *

import numpy as np
from scipy.interpolate import interp2d
import pandas as pd 

from DoE_package import *
from pyomo.contrib.sensitivity_toolbox.sens import sipopt

## Define Reaction Example Mathematical Model

Consider two chemical reactions that converts molecule $A$ to desired product $B$ and a less valuable side-product $C$.

$A \overset{k_1}{\rightarrow} B \overset{k_2}{\rightarrow} C$

Our ultimate goals is to design a large-scale continous reactor that maximizes the production of $B$. This general sequential reactions problem is widely applicable to CO$_2$ capture and industry more broadly (petrochemicals, pharmasuticals, etc.).

The rate laws for these two chemical reactions are:

$r_A = -k_1 C_A$

$r_B = k_1 C_A - k_2 C_B$

$r_C = k_2 C_B$

Here, $C_A$, $C_B$, and $C_C$ are the concentrations of each species. The rate constants $k_1$ and $k_2$ depend on temperature as follows:

$k_1 = A_1 \exp{\frac{-E_1}{R T}}$

$k_2 = A_2 \exp{\frac{-E_2}{R T}}$

$A_1, A_2, E_1$, and $E_2$ are fitted model parameters. $R$ is the ideal-gas constant and $T$ is absolute temperature.

Using the **CCSI$^2$ toolset**, we would like do the following perform:

Perform **uncertainty quantification** and **design of experiments** on a small-scale **batch reactor** to infer parameters $A_1$, $A_2$, $E_1$, and $E_2$.

### Batch reactor

The concenrations in a batch reactor evolve with time per the following differential equations:

$$ \frac{d C_A}{dt} = r_A = -k_1 C_A $$

$$ \frac{d C_B}{dt} = r_B = k_1 C_A - k_2 C_B $$

$$ \frac{d C_C}{dt} = r_C = k_2 C_B $$

This is a linear system of differential equations. Assuming the feed is only species $A$, i.e., 

$$C_A(t=0) = C_{A0} \quad C_B(t=0) = 0 \quad C_C(t=0) = 0$$

When the temperature is constant, it leads to the following analytic solution:

$$C_A(t) = C_{A,0} \exp(-k_1 t)$$

$$C_B(t) = \frac{k_1}{k_2 - k_1} C_{A,0} \left[\exp(-k_1 t) - \exp(-k_2 t) \right]$$

$$C_C(t) = C_{A,0} - \frac{k_2}{k_2 - k_1} C_{A,0} \exp(-k_1 t) + \frac{k_1}{k_2 - k_1} \exp(-k_2 t) C_{A,0} = C_{A,0} - C_{A}(t) - C_{B}(t)$$

In [2]:
from reactor_models import *

Three versions of this model is accomplished: 

Dynamic-DAE model: Temperature varying model discretized and integrated by Pyomo.DAE

Constant-DAE model: Temperature constant model discretized and integrated by Pyomo.DAE

Constant-analytical model: Temperature constant model discretized manually and using the analytical expressions for state variables.

In [3]:
# choose model from 'dynamic-DAE', 'constant-DAE', 'constant-analytical'
#model_opt = 'dynamic-DAE'
model_opt = 'constant-DAE'
#model_opt = 'constant-analytical'

if (model_opt=='dynamic-DAE'):
    createmod = create_model_dae
    disc = discretizer
    t_control = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
    
elif (model_opt=='constant-DAE'):
    createmod = create_model_dae_const
    disc = discretizer
    t_control = [0]
    
elif (model_opt=='constant-analytical'):
    createmod = create_model_alge
    disc = None
    t_control = [0]

    
# design variable and its control time set
dv_pass = {'CA0': [0],'T': t_control}
    
# Define measurement time points
t_measure = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]

In [4]:
# Define parameter nominal value 
parameter_dict = {'A1': 84.79085853498033, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}

def generate_exp(t_set, CA0, T):  
    '''Generate experiments. 
    t_set: time control set for T.
    CA0: CA0 value
    T: A list of T 
    '''
    assert(len(t_set)==len(T)), 'T should have the same length as t_set'
    
    T_con_initial = {}
    for t, tim in enumerate(t_set):
        T_con_initial[tim] = T[t]
        
    dv_dict_overall = {'CA0': {0: CA0},'T': T_con_initial}
    return dv_dict_overall

In [5]:
# empty prior
#prior_all = np.zeros((4,4))


# add prior information
prior_5_300 = pd.read_csv('fim_5_300_scale.csv')
prior_5_300_500 = pd.read_csv('fim_5_300_500_scale.csv')

prior_all = prior_5_300_500


prior_pass=np.asarray(prior_all)

#L_initials = np.linalg.cholesky(prior_pass)
#print(L_initials)

print('The prior information FIM:', prior_pass)
print('Prior Det:', np.linalg.det(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigvals(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigh(prior_pass)[1])

The prior information FIM: [[  28.67892806    5.41249739  -81.73674601  -24.02377324]
 [   5.41249739   26.40935036  -12.41816477 -139.23992532]
 [ -81.73674601  -12.41816477  240.46276004   58.76422806]
 [ -24.02377324 -139.23992532   58.76422806  767.25584508]]
Prior Det: 140189.67370995492
Eigenvalue of the prior experiments FIM: [8.00463185e+02 2.60471503e+02 4.84545259e-01 1.38765050e+00]
Eigenvalue of the prior experiments FIM: [[-0.77629273 -0.54211788  0.31871243 -0.0435912 ]
 [ 0.56614148 -0.80478003 -0.01425817 -0.177791  ]
 [-0.25919275 -0.19059651 -0.94009108  0.11278677]
 [ 0.09834911 -0.14869526  0.12019924  0.97661114]]


## Compute FIM 

This method computes an MBDoE optimization problem with no Degree of Freedom.

In [ ]:
# choose from 'simultaneous', 'sequential', 'sipopt'
#sensi_opt = 'simultaneous_finite'
#sensi_opt = 'sequential_finite'
sensi_opt = 'sequential_sipopt'
#sensi_opt = 'sequential_kaug'

# Define experiments
if (model_opt=='dynamic-DAE'):
    exp1 = generate_exp(t_control, 5, [570.21, 300, 300, 300, 300, 300, 300, 300, 300])
else: 
    exp1 = generate_exp(t_control, 5, [500])

print('Design variable:', exp1)

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 ['CA','CB','CC'], t_measure, createmod,
                                prior_FIM=prior_pass, discretize_model=disc)


if_s = True
result = doe_object.compute_FIM(exp1,mode=sensi_opt, FIM_store_name = 'dynamic.csv', scale_nominal_param_value=if_s, formula='central')


if sensi_opt == 'simultaneous_finite':
    result.extract_FIM(doe_object.m, doe_object.design_timeset,
                                                          doe_object.square_result, doe_object.obj_opt)
else:
    result.calculate_FIM(doe_object.jac, doe_object.design_values)

In [ ]:
print('======Result summary======')
print('Four design criteria log10() value:')
print('A-optimality:', np.log10(result.trace))
print('D-optimality:', np.log10(result.det))
print('E-optimality:', np.log10(result.min_eig))
print('Modified E-optimality:', np.log10(result.cond))

### Run sequential DOE automatically

This method runs a series of experiments in a row.

In [ ]:
exp1 = generate_exp(t_control, 5, [300])
exp2 = generate_exp(t_control, 5, [500.92])
exp3 = generate_exp(t_control, 5, [528])
exp4 = generate_exp(t_control, 5, [546.8])
exp5 = generate_exp(t_control, 5, [562])

In [ ]:

exp_all = [exp1, exp2, exp3, exp4, exp5]


doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 ['CA','CB','CC'], t_measure, createmod,
                                prior_FIM=prior_pass, discretize_model=disc)

# choose from 'simultaneous', 'sequential', 'sipopt'
#sensi_opt = 'simultaneous_finite'
sensi_opt = 'sequential_finite'
#sensi_opt = 'sequential_sipopt'
if_s = True
result = doe_object.sequential_exp(exp_all, mode=sensi_opt, tee_option=True,
                                    scale_nominal_param_value=if_s, formula='central')

In [ ]:
print('=======Result summary=======')
for i in range(len(exp_all)):
    print('The ', i+1,'th experiment has optimality log10() values:')
    print('A-optimality:', np.log10(result[0][i].trace))
    print('D-optimality:', np.log10(result[0][i].det))
    print('E-optimality:', np.log10(result[0][i].min_eig))
    print('Modified E-optimality:', np.log10(result[0][i].cond))

### Run 1D analysis

Run 1D sensitivity analysis with one changing design variable

Note: this method can only deal with constant T problems

In [ ]:
# Define experiments
if (model_opt=='dynamic-DAE'):
    exp1 = generate_exp(t_control, 5, [570.21, 300, 300, 300, 300, 300, 300, 300, 300])
else: 
    exp1 = generate_exp(t_control, 5, [500])


In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 ['CA','CB','CC'], t_measure, createmod,
                                prior_FIM=prior_pass, discretize_model=disc)

# choose from 'simultaneous', 'sequential', 'sipopt'
#sensi_opt = 'simultaneous_finite'
#sensi_opt = 'sequential_finite'
sensi_opt = 'sequential_sipopt'
if_s = True
result = doe_object.sensitivity_analysis_1D(exp1, {'T':[688,690]}, sensitivity_step=0.4, compare_opt='ME',
                                            mode=sensi_opt, tee_option=False,
                                    scale_nominal_param_value=if_s, formula='central')

In [ ]:
vary_range = list(result.keys())
result_list = list(result.values())

index_max = result_list.index(max(result_list))
index_min = result_list.index(min(result_list))

print('=======Result summary=======')
print('The biggest criterion value is:', max(result_list), ' at:', vary_range[index_max])
print('The smallest criterion value is:', min(result_list), ' at:', vary_range[index_min])

plt.plot(vary_range, result_list)
plt.xlabel('T [K]')
plt.ylabel('Design criteria')

## Optimization

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass, 
                                 ['CA','CB','CC'], t_measure, createmod,
                                 prior_FIM=prior_pass, discretize_model=disc)

square_result, optimize_result = doe_object.optimize_doe(exp1, if_optimize=True, if_cho=True, scale_nominal_param_value=True, obj_opt='det', L_initial=None)


In [ ]:
print('======Result summary======')
print('This optimization is solved with status:', optimize_result.status)
print('It gives solution:', optimize_result.solution)
print('The log10(OF) optimized is:', optimize_result.obj_value)
print('The result FIM is:', optimize_result.FIM)

## heatmap 

In [6]:
#design_ranges = {'CA0':list(np.linspace(1,5,9)),  'T': list(np.linspace(300,700,9))}
design_ranges = [list(np.linspace(1,5,9)),  list(np.linspace(300,700,9)), [300,500]]

#dv_apply_time = {'CA0': [0], 'T': t_control}
dv_apply_name = ['CA0', 'T', 'T']
dv_apply_time = [[0], [0], [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875,1]]

# Define experiments
if (model_opt=='dynamic-DAE'):
    exp1 = generate_exp(t_control, 5, [570.21, 300, 300, 300, 300, 300, 300, 300, 300])
else: 
    exp1 = generate_exp(t_control, 5, [500])

In [ ]:
a = {'1': [5, 4], '66':[7,8]}
b = a.copy()
print(b)
b['66'][1] = 4
print(b)

In [ ]:
from itertools import product 

dv_list = [list(np.linspace(1,5,9)),  list(np.linspace(300,700,9)), [300]]

#x = product(list(np.linspace(1,5,9)), list(np.linspace(300,700,9)), [5,6])
x = product(*dv_list)

print(x)

for i in x:
    print(list(i))

In [7]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass, 
                                 ['CA','CB','CC'], t_measure, createmod,
                                 prior_FIM=prior_pass, discretize_model=disc)

all_fim = doe_object.run_grid_search(exp1, design_ranges,dv_apply_name, dv_apply_time, 
                                     mode='sequential_finite', 
                                    scale_nominal_param_value=True
                                    )

Sensitivity information is scaled by its corresponding parameter nominal value.
(1.0, 300.0, 300)
0
[0]
1
[0]
2
[0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
=======This is the  1 th iteration=======
Design variable values of this iteration: {'CA0': {0: 1.0}, 'T': {0: 300.0, 0.125: 300, 0.25: 300, 0.375: 300, 0.5: 300, 0.625: 300, 0.75: 300, 0.875: 300, 1: 300}}
Sensitivity information is scaled by its corresponding parameter nominal value.
Existed information has been added.
FIM: [[  29.58003272    5.48612156  -84.54627162  -24.46790912]
 [   5.48612156   27.13328151  -12.64771469 -143.60704229]
 [ -84.54627162  -12.64771469  249.22249067   60.14898186]
 [ -24.46790912 -143.60704229   60.14898186  793.6004921 ]]
Trace: 1099.5362969977725
Determinant: 151638.4943213679
Condition number: 1703.5708820968514
Minimal eigen value: 0.48581739042109506
Eigen values: [8.27624360e+02 2.70029455e+02 4.85817390e-01 1.39666420e+00]
Eigen vectors: [[ 0.04300749 -0.31822906 -0.77652374  0.5421176

Existed information has been added.
FIM: [[  29.01044333    5.6006748   -82.42582703  -24.78056053]
 [   5.6006748    26.92260118  -12.8093067  -141.30405073]
 [ -82.42582703  -12.8093067   241.89507006   60.33727148]
 [ -24.78056053 -141.30405073   60.33727148  775.5570763 ]]
Trace: 1073.3851908623308
Determinant: 152189.04419265324
Condition number: 1609.9994023835127
Minimal eigen value: 0.5028391663221075
Eigen values: [8.09570757e+02 2.61884047e+02 5.02839166e-01 1.42754706e+00]
Eigen vectors: [[ 0.04433757 -0.31929264 -0.77731938  0.54024159]
 [ 0.17845201  0.014419    0.56435213  0.80588687]
 [-0.1142325   0.93968376 -0.2600425   0.19058663]
 [-0.97628889 -0.12181434  0.09828098  0.14953969]]
This is the  8  run out of  162 run.
The code has run 9.6194 seconds.
Estimated remaining time: 185.1740 seconds
(1.0, 500.0, 300)
0
[0]
1
[0]
2
[0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
=======This is the  9 th iteration=======
Design variable values of this iteration: {'CA0': {0: 1

Existed information has been added.
FIM: [[  28.80018525    5.49300764  -81.91123719  -24.24793267]
 [   5.49300764   26.52345436  -12.53402042 -139.55761761]
 [ -81.91123719  -12.53402042  240.71385582   59.08679739]
 [ -24.24793267 -139.55761761   59.08679739  768.1403749 ]]
Trace: 1064.1778703392624
Determinant: 147983.78531466573
Condition number: 1619.2882874718866
Minimal eigen value: 0.49498422928574204
Eigen values: [8.01522165e+02 2.60730128e+02 4.94984229e-01 1.43059273e+00]
Eigen vectors: [[ 0.04391558 -0.31898969 -0.77249369  0.54733034]
 [ 0.17798693  0.01419347  0.57166177  0.80082584]
 [-0.11327681  0.93992205 -0.25802797  0.19270827]
 [-0.97650419 -0.12079184  0.09938759  0.14822618]]
This is the  15  run out of  162 run.
The code has run 18.8551 seconds.
Estimated remaining time: 184.7795 seconds
(1.0, 650.0, 500)
0
[0]
1
[0]
2
[0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
=======This is the  16 th iteration=======
Design variable values of this iteration: {'CA0': {

Existed information has been added.
FIM: [[  30.12752784    6.12805011  -85.60807009  -27.7236819 ]
 [   6.12805011   29.19744073  -14.3304499  -153.65631904]
 [ -85.60807009  -14.3304499   250.80871635   68.65208131]
 [ -27.7236819  -153.65631904   68.65208131  841.79876875]]
Trace: 1151.9324536678646
Determinant: 164685.70018609465
Condition number: 1771.4801724742997
Minimal eigen value: 0.4964321047417337
Eigen values: [8.79419631e+02 2.70622477e+02 4.96432105e-01 1.39391410e+00]
Eigen vectors: [[ 0.04491711 -0.31961785 -0.77643054  0.54127858]
 [ 0.17866548  0.01524896  0.5653439   0.8051288 ]
 [-0.11677295  0.93923424 -0.26030484  0.19090448]
 [-0.97592273 -0.12430185  0.09891048  0.14946769]]
This is the  22  run out of  162 run.
The code has run 27.0056 seconds.
Estimated remaining time: 171.8538 seconds
(1.5, 400.0, 300)
0
[0]
1
[0]
2
[0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
=======This is the  23 th iteration=======
Design variable values of this iteration: {'CA0': {0

Existed information has been added.
FIM: [[  29.10620604    5.67772258  -82.46339932  -24.89648593]
 [   5.67772258   26.93219884  -12.86922226 -140.9603365 ]
 [ -82.46339932  -12.86922226  241.69854807   60.24841433]
 [ -24.89648593 -140.9603365    60.24841433  772.91678654]]
Trace: 1070.65373948992
Determinant: 164251.619804841
Condition number: 1551.5407215360776
Minimal eigen value: 0.5200809843403219
Eigen values: [8.06926826e+02 2.61711348e+02 5.20080984e-01 1.49548471e+00]
Eigen vectors: [[ 0.04470411 -0.31962802 -0.77317125  0.54593561]
 [ 0.17863376  0.01423995  0.57042457  0.80156286]
 [-0.11464267  0.93950088 -0.25874541  0.19299216]
 [-0.97619085 -0.12236526  0.09936214  0.14901452]]
This is the  29  run out of  162 run.
The code has run 35.5636 seconds.
Estimated remaining time: 163.1019 seconds
(1.5, 550.0, 500)
0
[0]
1
[0]
2
[0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
=======This is the  30 th iteration=======
Design variable values of this iteration: {'CA0': {0: 1.

...

Existed information has been added.
FIM: [[  35.39211234    9.22308995  -95.69063657  -39.3487156 ]
 [   9.22308995   36.80267948  -20.33878885 -181.038465  ]
 [ -95.69063657  -20.33878885  269.4670378    90.6183572 ]
 [ -39.3487156  -181.038465     90.6183572   935.35577706]]
Trace: 1277.017606694349
Determinant: 478375.47935399483
Condition number: 1199.9989788371267
Minimal eigen value: 0.8212252566049608
Eigen values: [9.85469469e+02 2.88679300e+02 8.21225257e-01 2.04761261e+00]
Eigen vectors: [[ 0.05571542 -0.32911178 -0.7844296   0.52273457]
 [ 0.18875266  0.01745765  0.54869009  0.81425232]
 [-0.13569991  0.93279596 -0.27146248  0.19438454]
 [-0.97100657 -0.14585045  0.09958669  0.16110983]]
This is the  134  run out of  162 run.
The code has run 178.8289 seconds.
Estimated remaining time: 37.3672 seconds
(4.5, 500.0, 300)
0
[0]
1
[0]
2
[0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
=======This is the  135 th iteration=======
Design variable values of this iteration: {'CA0': {

Existed information has been added.
FIM: [[  31.13438628    7.04282989  -85.27019232  -28.5630018 ]
 [   7.04282989   28.71995645  -14.7642416  -145.67319431]
 [ -85.27019232  -14.7642416   245.54744961   65.29625692]
 [ -28.5630018  -145.67319431   65.29625692  785.16757397]]
Trace: 1090.569366309676
Determinant: 334072.2908212103
Condition number: 1201.6870700885588
Minimal eigen value: 0.6840342314306751
Eigen values: [8.21995091e+02 2.65653696e+02 6.84034231e-01 2.23654444e+00]
Eigen vectors: [[ 0.05001058 -0.32412002 -0.72914289  0.6006628 ]
 [ 0.18166178  0.01308144  0.62914363  0.7556495 ]
 [-0.12242742  0.93668688 -0.24550229  0.21761858]
 [-0.97442776 -0.1318817   0.1107139   0.14436129]]
This is the  141  run out of  162 run.
The code has run 188.9469 seconds.
Estimated remaining time: 28.1410 seconds
(4.5, 650.0, 500)
0
[0]
1
[0]
2
[0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
=======This is the  142 th iteration=======
Design variable values of this iteration: {'CA0': {0

Existed information has been added.
FIM: [[  44.77448121   13.36308321 -124.75145794  -65.13386954]
 [  13.36308321   57.3881323   -33.66577731 -299.42207777]
 [-124.75145794  -33.66577731  355.41783016  168.62926416]
 [ -65.13386954 -299.42207777  168.62926416 1595.51055227]]
Trace: 2053.0909959474125
Determinant: 513228.2976445542
Condition number: 2917.190381044787
Minimal eigen value: 0.5753167063296618
Eigen values: [1.67830836e+03 3.72781455e+02 5.75316706e-01 1.42586218e+00]
Eigen vectors: [[-0.05047007  0.32648857 -0.77233623  0.54254469]
 [-0.18287382 -0.02050882  0.56719727  0.80276012]
 [ 0.13340808 -0.93436539 -0.26673157  0.1949817 ]
 [ 0.9727344   0.14122999  0.10314205  0.15232724]]
This is the  148  run out of  162 run.
The code has run 198.4624 seconds.
Estimated remaining time: 18.7735 seconds
(5.0, 400.0, 300)
0
[0]
1
[0]
2
[0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
=======This is the  149 th iteration=======
Design variable values of this iteration: {'CA0': {0

Existed information has been added.
FIM: [[  33.42646128    8.35944401  -89.81066931  -33.72058097]
 [   8.35944401   32.21877784  -17.42991033 -158.35560484]
 [ -89.81066931  -17.42991033  254.19373028   75.25517256]
 [ -33.72058097 -158.35560484   75.25517256  830.15519359]]
Trace: 1149.9941629861146
Determinant: 513459.05664163764
Condition number: 1009.1034533531276
Minimal eigen value: 0.8647458183914464
Eigen values: [8.72617992e+02 2.74028305e+02 8.64745818e-01 2.48312030e+00]
Eigen vectors: [[ 0.0549935  -0.32812852 -0.75602079  0.56368427]
 [ 0.18644071  0.01443566  0.59187188  0.78404028]
 [-0.13153067  0.93376588 -0.2587196   0.2093923 ]
 [-0.97206752 -0.14214279  0.10575638  0.15393425]]
This is the  155  run out of  162 run.
The code has run 208.4038 seconds.
Estimated remaining time: 9.4118 seconds
(5.0, 550.0, 500)
0
[0]
1
[0]
2
[0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
=======This is the  156 th iteration=======
Design variable values of this iteration: {'CA0': {

Existed information has been added.
FIM: [[  31.06754308    6.93261012  -84.9284853   -27.95381436]
 [   6.93261012   28.24957649  -14.44938555 -143.99757491]
 [ -84.9284853   -14.44938555  244.72765813   64.01566832]
 [ -27.95381436 -143.99757491   64.01566832  779.5560886 ]]
Trace: 1083.6008663096845
Determinant: 330586.4905784959
Condition number: 1212.388775069196
Minimal eigen value: 0.6727727401066618
Eigen values: [8.15662118e+02 2.64992592e+02 6.72772740e-01 2.27338343e+00]
Eigen vectors: [[ 0.04944936 -0.32415168 -0.70688843  0.62672897]
 [ 0.18091586  0.01275908  0.65620673  0.73246119]
 [-0.12122713  0.93686107 -0.23742408  0.22632968]
 [-0.9747452  -0.13059183  0.11546121  0.1395933 ]]
This is the  162  run out of  162 run.
The code has run 218.9364 seconds.
Estimated remaining time: 0.0000 seconds
The whole run takes  218.93642497062683  s.


In [8]:
heatmap = Grid_Search_Result(design_ranges, all_fim, store_optimality_name='test_heatmap.csv')

heatmap.heatmap('Reactor case', '$C_{A0}$ [M]', 'T [K]')

AttributeError: 'list' object has no attribute 'keys'